<a href="https://colab.research.google.com/github/saiteja-d/spark_course/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!ls

sample_data  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz


In [8]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [959 kB]
Get:15 http://archive.ub

In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [16]:
!ls


sample_data		   spark-2.3.1-bin-hadoop2.7.tgz
spark-2.3.1-bin-hadoop2.7  spark-2.3.1-bin-hadoop2.7.tgz.1


In [34]:
!wget https://raw.githubusercontent.com/sai-usf/files/master/clustering_dataset.csv

--2019-04-02 22:53:18--  https://raw.githubusercontent.com/sai-usf/files/master/clustering_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694 [text/plain]
Saving to: ‘clustering_dataset.csv.1’

clustering_dataset. 100%[===================>]     694  --.-KB/s    in 0s      

2019-04-02 22:53:18 (73.4 MB/s) - ‘clustering_dataset.csv.1’ saved [694/694]



In [0]:
mv clustering_dataset.csv.1 dataset.csv

In [30]:
!ls

dataset.csv  spark-2.3.1-bin-hadoop2.7	    spark-2.3.1-bin-hadoop2.7.tgz.1
sample_data  spark-2.3.1-bin-hadoop2.7.tgz  spark-warehouse


In [0]:
from pyspark.ml.linalg import Vectors

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
from pyspark.ml.feature import VectorAssembler

#Loading Data

In [0]:
rc = spark.read.csv('dataset.csv',header=True, inferSchema=True)

In [134]:
rc

DataFrame[col1: int, col2: int, col3: int]

In [135]:
rc.printSchema()

root
 |-- col1: integer (nullable = true)
 |-- col2: integer (nullable = true)
 |-- col3: integer (nullable = true)



In [0]:
rc.write.parquet("proto1.parquet")

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# to read parquet file
df = sqlContext.read.parquet('proto1.parquet')

In [137]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[col1: int, col2: int, col3: int]>

#Transformations

In [0]:
vectorAssembler = VectorAssembler(inputCols=["col1","col2","col3"], outputCol="features")

In [0]:
vcluster_df = vectorAssembler.transform(rc)

In [140]:
vcluster_df.show()

+----+----+----+--------------+
|col1|col2|col3|      features|
+----+----+----+--------------+
|   7|   4|   1| [7.0,4.0,1.0]|
|   7|   7|   9| [7.0,7.0,9.0]|
|   7|   9|   6| [7.0,9.0,6.0]|
|   1|   6|   5| [1.0,6.0,5.0]|
|   6|   7|   7| [6.0,7.0,7.0]|
|   7|   9|   4| [7.0,9.0,4.0]|
|   7|  10|   6|[7.0,10.0,6.0]|
|   7|   8|   2| [7.0,8.0,2.0]|
|   8|   3|   8| [8.0,3.0,8.0]|
|   4|  10|   5|[4.0,10.0,5.0]|
|   7|   4|   5| [7.0,4.0,5.0]|
|   7|   8|   4| [7.0,8.0,4.0]|
|   2|   5|   1| [2.0,5.0,1.0]|
|   2|   6|   2| [2.0,6.0,2.0]|
|   2|   3|   8| [2.0,3.0,8.0]|
|   3|   9|   1| [3.0,9.0,1.0]|
|   4|   2|   9| [4.0,2.0,9.0]|
|   1|   7|   1| [1.0,7.0,1.0]|
|   6|   2|   3| [6.0,2.0,3.0]|
|   4|   1|   9| [4.0,1.0,9.0]|
+----+----+----+--------------+
only showing top 20 rows



#Kmeans


In [0]:
kmeans = KMeans().setK(3)

In [0]:
kmeans = kmeans.setSeed(1)

In [0]:
kmodel = kmeans.fit(vcluster_df)

#Find Centers

In [144]:
centers = kmodel.clusterCenters()
centers

[array([35.88461538, 31.46153846, 34.42307692]),
 array([5.12, 5.84, 4.84]),
 array([80.        , 79.20833333, 78.29166667])]

In [0]:
from pyspark.ml.clustering import BisectingKMeans


In [0]:
bkmeans = BisectingKMeans().setK(3)
bkmeans = bkmeans.setSeed(1)

In [0]:
bkmodel = bkmeans.fit(vcluster_df)

In [150]:
bkcenters = bkmodel.clusterCenters()
bkcenters

[array([5.12, 5.84, 4.84]),
 array([35.88461538, 31.46153846, 34.42307692]),
 array([80.        , 79.20833333, 78.29166667])]